In [1]:
# should already be activated, but just a reminder where ya are :)  
# !conda activate wool_sucking_nn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

%matplotlib inline

In [3]:
# import data
excel_file = pd.read_excel('..\Data\cleaned_and_split_data.xlsx', sheet_name=None)

# Unpack the dataframes into separate variables
X_train = excel_file['X_train']
X_test = excel_file['X_test']
y_train = excel_file['y_train']
y_test = excel_file['y_test']

In [4]:
print(X_train.shape)
X_train.head()

(3965, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,2,4,4,3
1,1,HCS,1,2,4,4,2
2,0,ORI,1,1,1,1,1
3,1,MCO,1,1,2,2,3
4,1,HCS,1,1,2,1,2


In [5]:
print(X_test.shape)
X_test.head()

(1700, 7)


,Neuter_status,Breed_group,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming
0,1,SIB,1,3,3,1,1
1,1,MCO,1,1,2,1,1
2,1,RUS,1,1,4,4,1
3,0,EUR,2,2,2,2,1
4,1,NFO,1,2,3,2,1


In [6]:
print(y_train.shape)
y_train.head()

(3965, 1)


,Wool_sucking_binary
0,0
1,0
2,0
3,0
4,0


In [7]:
print(y_test.shape)
y_test.head()

(1700, 1)


,Wool_sucking_binary
0,1
1,1
2,1
3,0
4,1


# Prep for tensorflow.keras

In [38]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [9]:
# one-hot breed 
X_train_encoded = pd.get_dummies(X_train, columns=['Breed_group'])
X_test_encoded = pd.get_dummies(X_test, columns=['Breed_group'])

In [10]:
X_train_encoded

,Neuter_status,Aggression_owner,Aggression_cats,Shyness_novel,Shyness_strangers,Grooming,Breed_group_ABY,Breed_group_BEN,Breed_group_BRI,Breed_group_BUR,...,Breed_group_MCO,Breed_group_NFO,Breed_group_ORI,Breed_group_PER,Breed_group_RAG,Breed_group_RUS,Breed_group_SBI,Breed_group_SIB,Breed_group_TUV,Breed_group_other
0,1,1,2,4,4,3,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,1,2,4,4,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,1,2,2,3,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,1,1,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3960,1,1,3,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3961,1,2,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3962,1,1,1,3,4,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3963,1,1,1,4,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Plain Jane

In [11]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [18]:
def recall(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

tf.keras.utils.get_custom_objects()['recall'] = recall

In [19]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [20]:
model.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 0s 868us/step - loss: 0.5764 - accuracy: 0.7006 - recall: 0.2191
Epoch 2/10
124/124 [==============================] - 0s 884us/step - loss: 0.5761 - accuracy: 0.7014 - recall: 0.2148
Epoch 3/10
124/124 [==============================] - 0s 892us/step - loss: 0.5732 - accuracy: 0.7084 - recall: 0.2436
Epoch 4/10
124/124 [==============================] - 0s 884us/step - loss: 0.5693 - accuracy: 0.7095 - recall: 0.2449
Epoch 5/10
124/124 [==============================] - 0s 884us/step - loss: 0.5695 - accuracy: 0.7049 - recall: 0.2246
Epoch 6/10
124/124 [==============================] - 0s 957us/step - loss: 0.5685 - accuracy: 0.7067 - recall: 0.2559
Epoch 7/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5659 - accuracy: 0.7082 - recall: 0.2447
Epoch 8/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5630 - accuracy: 0.7120 - recall: 0.2628
Epoch 9/10
124/124 [==============================] 

In [22]:
test_loss, test_acc, test_recall = model.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 715us/step - loss: 0.5921 - accuracy: 0.6847 - recall: 0.1780
Test Accuracy: 0.6847058534622192
Test Loss: 0.5921450853347778
Test Recall: 0.17800509929656982


In [26]:
y_pred = model.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
cm

54/54 [==============================] - 0s 583us/step


array([[1066,   93],
       [ 443,   98]], dtype=int64)

Not doing too hot, but also seems to be doing better than initial models in Cat-pstone 1. 

## Adagrad optimizer

In [29]:
model2 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(len(X_train_encoded.columns),)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model2.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

In [35]:
model2.fit(X_train_encoded, y_train, epochs=10, batch_size=32)

Epoch 1/10
124/124 [==============================] - 0s 884us/step - loss: 0.5652 - accuracy: 0.7092 - recall: 0.2324
Epoch 2/10
124/124 [==============================] - 0s 884us/step - loss: 0.5631 - accuracy: 0.7095 - recall: 0.2274
Epoch 3/10
124/124 [==============================] - 0s 884us/step - loss: 0.5605 - accuracy: 0.7127 - recall: 0.2465
Epoch 4/10
124/124 [==============================] - 0s 884us/step - loss: 0.5606 - accuracy: 0.7082 - recall: 0.2393
Epoch 5/10
124/124 [==============================] - 0s 965us/step - loss: 0.5598 - accuracy: 0.7107 - recall: 0.2398
Epoch 6/10
124/124 [==============================] - 0s 1ms/step - loss: 0.5593 - accuracy: 0.7115 - recall: 0.2369
Epoch 7/10
124/124 [==============================] - 0s 989us/step - loss: 0.5585 - accuracy: 0.7137 - recall: 0.2457
Epoch 8/10
124/124 [==============================] - 0s 989us/step - loss: 0.5580 - accuracy: 0.7145 - recall: 0.2554
Epoch 9/10
124/124 [==============================

In [36]:
test_loss, test_acc, test_recall = model2.evaluate(X_test_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

54/54 [==============================] - 0s 753us/step - loss: 0.5886 - accuracy: 0.6771 - recall: 0.2351
Test Accuracy: 0.6770588159561157
Test Loss: 0.5885889530181885
Test Recall: 0.23510226607322693


In [37]:
y_pred = model2.predict(X_test_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
cm

54/54 [==============================] - 0s 640us/step


array([[1018,  141],
       [ 408,  133]], dtype=int64)

## Scaled with Plain Jane and Adagrad

In [39]:
scaler = StandardScaler()
scaler.fit(X_train_encoded)

StandardScaler()

In [40]:
X_train_scaled_encoded = scaler.transform(X_train_encoded)
X_test_scaled_encoded = scaler.transform(X_test_encoded)

In [42]:
X_train_scaled_encoded.shape

(3965, 25)

In [44]:
# Plain Jane + scaled
model3 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model3.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model3.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model3.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model3.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 900us/step - loss: 0.6301 - accuracy: 0.6560 - recall: 0.0964
Epoch 2/10
124/124 [==============================] - 0s 932us/step - loss: 0.5881 - accuracy: 0.6943 - recall: 0.1694
Epoch 3/10
124/124 [==============================] - 0s 941us/step - loss: 0.5833 - accuracy: 0.6951 - recall: 0.1673
Epoch 4/10
124/124 [==============================] - 0s 932us/step - loss: 0.5788 - accuracy: 0.6948 - recall: 0.2100
Epoch 5/10
124/124 [==============================] - 0s 916us/step - loss: 0.5733 - accuracy: 0.7014 - recall: 0.2227
Epoch 6/10
124/124 [==============================] - 0s 932us/step - loss: 0.5733 - accuracy: 0.7026 - recall: 0.2124
Epoch 7/10
124/124 [==============================] - 0s 949us/step - loss: 0.5683 - accuracy: 0.7064 - recall: 0.2487
Epoch 8/10
124/124 [==============================] - 0s 924us/step - loss: 0.5646 - accuracy: 0.7095 - recall: 0.2646
Epoch 9/10
124/124 [============================

array([[1005,  154],
       [ 405,  136]], dtype=int64)

In [46]:
# adagrad + scaled 
model4 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(25,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Adagrad better for when feataures vary in magnitude and frequency 
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model4.compile(optimizer=optimizer, 
             loss='binary_crossentropy',
             metrics=['accuracy', 'recall'])

model4.fit(X_train_scaled_encoded, y_train, epochs=10, batch_size=32)

test_loss, test_acc, test_recall = model4.evaluate(X_test_scaled_encoded, y_test)
print('Test Accuracy:', test_acc)
print('Test Loss:', test_loss)
print('Test Recall:', test_recall)

y_pred = model4.predict(X_test_scaled_encoded)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
cm

Epoch 1/10
124/124 [==============================] - 0s 868us/step - loss: 0.6272 - accuracy: 0.6739 - recall: 0.0338
Epoch 2/10
124/124 [==============================] - 0s 900us/step - loss: 0.6017 - accuracy: 0.6820 - recall: 0.0467
Epoch 3/10
124/124 [==============================] - 0s 884us/step - loss: 0.5941 - accuracy: 0.6852 - recall: 0.0888
Epoch 4/10
124/124 [==============================] - 0s 835us/step - loss: 0.5898 - accuracy: 0.6910 - recall: 0.1193
Epoch 5/10
124/124 [==============================] - 0s 819us/step - loss: 0.5868 - accuracy: 0.6936 - recall: 0.1359
Epoch 6/10
124/124 [==============================] - 0s 827us/step - loss: 0.5843 - accuracy: 0.6974 - recall: 0.1593
Epoch 7/10
124/124 [==============================] - 0s 916us/step - loss: 0.5822 - accuracy: 0.6989 - recall: 0.1675
Epoch 8/10
124/124 [==============================] - 0s 884us/step - loss: 0.5809 - accuracy: 0.7016 - recall: 0.2046
Epoch 9/10
124/124 [============================

array([[1084,   75],
       [ 459,   82]], dtype=int64)